조엘 그루스, 2016, 인사이트, '밑바닥부터 시작하는 데이터 과학'

범주형 결과를 반환하는 분류나무 - classification tree<br>
숫자형 결과를 반환하는 회귀나무 - regression tree

In [1]:
from collections import Counter, defaultdict
from functools import partial
import math, random
후보자_합격여부 = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

### ID3알고리즘을 기반으로 분류나무 생성

In [2]:
def class_probabilities(labels):
    '''클래스(라벨)에 속할 확률'''
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

#엔트로피 == 무질서도, 무질서도가 낮은 방향으로 파티션을 나눈다.
def entropy(class_probabilities):
    """각 클래스에 속할 확률로부터의 각각의 엔트로피 총합"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def data_entropy(labeled_data):
    '''데이터 전체의 엔트로피 계산'''
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    """각 파티션들로부터의 엔트로피 계산 후 총합"""
    total_count = sum(len(subset) for subset in subsets)
    return sum( data_entropy(subset) * len(subset) / total_count # 각 파티션의 엔트로피를 비율(파티션길이/전체)로 가중합한다. 
                for subset in subsets )

In [3]:
def partition_by(inputs, attribute):
    """attribute에 따라 inputs의 파티션을 나눈다"""
    groups = defaultdict(list)
    for input_ in inputs:
        key = input_[0][attribute] #attribute에 해당하는 각 데이터의 값을 가져오기
        groups[key].append(input_) #데이터 값에 따른 데이터를 저장 - 파티션으로 나누는 것
    return groups    

def partition_entropy_by(inputs,attribute):
    """나눈 파티션으로 총 엔트로피 계산"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())    

In [4]:
# 처음 분류할 때 level이 가장 엔트로피가 낫은 구분이다.
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(후보자_합격여부, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


### 가장 낮은 엔트로피 Level로 파티션 나눈 예시

{'Senior': 
<ul><li>[({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'},False),</li>
    <li>({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False),<br></li>
    <li>({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False),<br></li>
    <li>({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),<br></li>
    <li>({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'},True)],<br></li></ul>
 'Mid': 
<ul><li>[({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'},True),</li>
    <li>({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, True),</li>
    <li>({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True),</li>
    <li>({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True)],</li></ul>
 'Junior': 
<ul><li>[({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True),</li>
    <li>({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),</li>
    <li>({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False),</li>
    <li>({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'no'}, True),</li>
    <li>({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, False)]}</li>

In [5]:
for attr in ['Senior','Mid','Junior']:
    senior_inputs = [(input_, label) for input_, label in 후보자_합격여부 if input_["level"] == attr]
    print('\n'+attr+"의 데이터에서 각 속성별 엔트로피")
    for key in ['lang','tweets','phd']:
        print(key, partition_entropy_by(senior_inputs, key))


Senior의 데이터에서 각 속성별 엔트로피
lang 0.4
tweets 0.0
phd 0.9509775004326938

Mid의 데이터에서 각 속성별 엔트로피
lang 0.0
tweets 0.0
phd 0.0

Junior의 데이터에서 각 속성별 엔트로피
lang 0.9509775004326938
tweets 0.9509775004326938
phd 0.0


In [6]:
def build_tree_id3(inputs, split_candidates=None):
    # 첫 분기라면 입력 데이터의 모든 변수가 분기 후보
    if split_candidates is None:
        # 'lang', 'tweets', 'phd', 'level' 모두 후보
        split_candidates = inputs[0][0].keys()

    # 입력 데이터에서 범주별 개수를 세어 본다
    num_inputs = len(inputs)
    num_class0 = len([label for _, label in inputs if label])
    num_class1 = num_inputs - num_class0

    # class0(true)이 하나도 없으면 False leaf 반환
    if num_class0 == 0: return False
    # class1(false)이 하나도 없으면 Ture leaf 반환
    if num_class1 == 0: return True

    # 파티션 기준으로 사용할 변수가 없다면
    if not split_candidates:
        # 다수결로 결정
        # class0(true)가 많으면 true,
        # class1(false)가 많으면 false 반환
        return num_class0 >= num_class1

    # 아니면 가장 적합한 변수를 기준으로 분기
    best_attribute = min(split_candidates,
                         key=partial(partition_entropy_by, inputs))
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # 재귀적으로 서브트리를 구축
    subtrees = { attribute_value : build_tree_id3(subset, new_candidates)
                 for attribute_value, subset in partitions.items()}
    # 기본값
    subtrees[None] = num_class0 > num_class1 
    return (best_attribute, subtrees)

In [7]:
tree = build_tree_id3(후보자_합격여부)
print(tree)

('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True}), None: True})


('level', {'Senior': ('tweets', {'no': False, 
                                 'yes': True, 
                                 None: False}), 
           'Mid': True, 
           'Junior': ('phd', 
                     {'no': True, 
                      'yes': False, 
                      None: True}), 
                      None: True})

In [8]:
def classify(tree, input):
    
    """마지막이면 True 또는 False 반환"""
    if tree in [True, False]:
        return tree
    
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  

    if subtree_key not in subtree_dict: 
        subtree_key = None              
    
    subtree = subtree_dict[subtree_key] 
    return classify(subtree, input)     

In [9]:
print(" Junior / Java / tweets / phd\n", \
      "결과:",classify(tree, { "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "yes"}))

 Junior / Java / tweets / phd
 결과: False


### 랜덤포레스트
여러개의 의사결정나무를 만든 후 다수결로 결과 결정
>1) 데이터를 bootstrap하여 랜덤성 부여<br>
>2) 속성 선택의 랜덤화 : <br>
         파티션을 나누는 속성을 랜덤으로 몇개만 한정한 뒤 그중에서 엔트로피 계산하여 선택한다

부트스트랩 : 동일데이터에서 복원추출로 샘플링

In [10]:
def bootstrap_sample(data):
    """randomly samples len(data) elements with replacement"""
    return [random.choice(data) for _ in data]
    
def bootstrap_statistic(data, stats_fn, num_samples):
    """evaluates stats_fn on num_samples bootstrap samples from data"""
    return [stats_fn(bootstrap_sample(data)) 
            for _ in range(num_samples)]

In [18]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

In [19]:
일번데이터 = bootstrap_sample(후보자_합격여부)
이번데이터 = bootstrap_sample(후보자_합격여부)
print("같은 데이터다" if 일번데이터 == 이번데이터 else "다른 데이터다")

다른 데이터다


In [20]:
forest = list()
for _ in range(5):
    forest.append(build_tree_id3(bootstrap_sample(후보자_합격여부)))

In [21]:
print(" Junior / Java / tweets / phd\n", \
      "결과:", forest_classify(forest,  { "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "yes"}))

 Junior / Java / tweets / phd
 결과: False


속성선택의 랜덤화

In [22]:
def build_tree_of_forest(inputs, split_candidates=None):
    num_split_candidates = 2
    # 첫 분기라면 입력 데이터의 모든 변수가 분기 후보
    if split_candidates is None:
        # 'lang', 'tweets', 'phd', 'level' 모두 후보
        split_candidates = inputs[0][0].keys()

    # 입력 데이터에서 범주별 개수를 세어 본다
    num_inputs = len(inputs)
    num_class0 = len([label for _, label in inputs if label])
    num_class1 = num_inputs - num_class0

    # class0(true)이 하나도 없으면 False leaf 반환
    if num_class0 == 0: return False
    # class1(false)이 하나도 없으면 Ture leaf 반환
    if num_class1 == 0: return True

    # 파티션 기준으로 사용할 변수가 없다면
    if not split_candidates:
        # 다수결로 결정
        # class0(true)가 많으면 true,
        # class1(false)가 많으면 false 반환
        return num_class0 >= num_class1
    
    if len(split_candidates) <= num_split_candidates:
        sampled_split_candidates = split_candidates
    else:
        sampled_split_candidates = random.sample(split_candidates, num_split_candidates)

    # 아니면 가장 적합한 변수를 기준으로 분기
    best_attribute = min(sampled_split_candidates,
                         key=partial(partition_entropy_by, inputs))
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # 재귀적으로 서브트리를 구축
    subtrees = { attribute_value : build_tree_of_forest(subset, new_candidates)
                 for attribute_value, subset in partitions.items()}
    # 기본값
    subtrees[None] = num_class0 > num_class1 
    return (best_attribute, subtrees)

In [23]:
일번트리 = build_tree_of_forest(후보자_합격여부)
이번트리 = build_tree_of_forest(후보자_합격여부)
print("같은 트리다" if 일번트리 == 이번트리 else "다른 트리다")

다른 트리다


In [24]:
forest_attr = list()
for _ in range(5):
    forest_attr.append(build_tree_of_forest(후보자_합격여부))

In [25]:
print(" Junior / Java / tweets / phd\n", \
      "결과:", forest_classify(forest_attr,  { "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "yes"}))

 Junior / Java / tweets / phd
 결과: False
